In [ ]:
import requests
from datetime import datetime
from transformers import pipeline
from dateutil import parser
import pandas as pd
import pickle
from flask import Flask, request, jsonify
from dateutil.tz import tz
import pytz
from flask_cors import CORS

app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Initialize sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# API URLs
NEWS_API_URL = "https://mboum-finance.p.rapidapi.com/v2/markets/news"
REAL_TIME_API_URL = "https://mboum-finance.p.rapidapi.com/v1/markets/quote"
HISTORY_API_URL = "https://mboum-finance.p.rapidapi.com/v1/markets/stock/quotes"

# API Headers with your RapidAPI key
headers = {
    "X-RapidAPI-Key": "",  # Replace with your RapidAPI key
    "X-RapidAPI-Host": "mboum-finance.p.rapidapi.com"
}

# Function to fetch today's financial news for a stock symbol
def get_today_financial_news(stock_symbol):
    params = {"tickers": stock_symbol}
    response = requests.get(NEWS_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching news: {response.status_code}")
        return None

# Function to fetch today's stock price for a stock symbol
def get_today_stock_price(stock_symbol):
    params = {"symbol": stock_symbol, "type": "STOCKS" }
    response = requests.get(REAL_TIME_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching stock price: {response.status_code}")
        return None

# Function to fetch historical stock data
def get_history_stock_price(stock_symbol):
    params = {"symbol": stock_symbol}
    response = requests.get(HISTORY_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching history data: {response.status_code}")
        return None

# Function to analyze sentiment and calculate sentiment score and confidence
def analyze_sentiment(news_data):
    sentiment_vector = []
    confidence_vector = []
    
    est = pytz.timezone('US/Eastern')

    # Get the current date and time in UTC, then convert to EST
    today_est = datetime.now(est)
    today_date = today_est.date()
    print("today_date: ", today_date)

    for article in news_data.get('body', []):
        article_date_str = article.get('time')  # e.g., "Dec 5, 2024, 4:40 AM EST"
        tzinfos = {"EST": tz.tzoffset("EST", -5 * 3600)}  # EST is UTC-5

        try:
            article_date = parser.parse(article_date_str, tzinfos=tzinfos)
        except ValueError as e:
            print(f"Error parsing date: {e}")
            continue

        print("article_date.date(): ", article_date.date())
        if article_date.date() != today_date:  # Only process today's articles
            continue

        title = article.get('title')
        
        if title:
            sentiment_result = sentiment_pipeline(title)[0]
            sentiment = sentiment_result['label']
            confidence = sentiment_result['score']
            sentiment_vector.append(sentiment)
            confidence_vector.append(confidence)
    
    return sentiment_vector, confidence_vector

# Function to calculate cumulative sentiment score and confidence sum
def calculate_sentiment_score(sentiment_vector, confidence_vector):
    sentiment_score = 0
    confidence_sum = 0
    
    for i in range(len(sentiment_vector)):
        if sentiment_vector[i] == 'positive':
            sentiment_score += 1
            confidence_sum += confidence_vector[i]
        elif sentiment_vector[i] == 'negative':
            sentiment_score -= 1
            confidence_sum -= confidence_vector[i]
    print("sentiment_score: ", sentiment_score)
    print("confidence_sum: ", confidence_sum) 
    return sentiment_score, confidence_sum

# Function to predict return
def predict_return(stock_symbol, date, open_price, volume, weighted_sentiment_score):
    """
    Predict return based on the inputs using the saved SARIMAX model.

    Args:
        stock_symbol (str): The stock symbol (e.g., 'AAPL', 'TSLA').
        date (str): The date for prediction (format: 'YYYY-MM-DD').
        open_price (float): The open price of the stock.
        volume (int): The volume of the stock.
        weighted_sentiment_score (float): Weighted sentiment score.

    Returns:
        float: Predicted return for the given inputs.
    """
    # Load the saved SARIMAX model for the stock
    model_path = f'/home/ec2-user/models/saved_models/{stock_symbol}_sarimax_model.pkl'
    try:
        with open(model_path, 'rb') as model_file:
            model = pickle.load(model_file)
    except FileNotFoundError:
        raise ValueError(f"Model for stock '{stock_symbol}' not found at {model_path}.")

    print("date: ", date)
    print("open_price: ", type(open_price))
#     volume = int(volume.replace(",", ""))

    print("volume: ", type(volume))
    print("weighted_sentiment_score: ", type(weighted_sentiment_score))
    # Prepare exogenous variables as a DataFrame
    exog = pd.DataFrame({
        'Date': [datetime.strptime(date, '%Y-%m-%d')],
        'open': [open_price],
        'volume': [volume],
        'weighted_sentiment_score': [weighted_sentiment_score]
    })
    
    print("exog: ", exog)

    # Predict the return using the loaded SARIMAX model
    predictions = model.get_forecast(steps=1, exog=exog[['open', 'volume', 'weighted_sentiment_score']]).predicted_mean

    # Extract the first prediction
    prediction = predictions.iloc[0]

    return prediction

def combine_recommendations(price_change, predicted_return, professional_recommendation, w_predicted=0.7, w_professional=0.3, return_threshold = 0.03):
    # Step 1: Convert predicted return to recommendation
    return_diff = predicted_return - price_change
    
    print("return_diff: ", return_diff)
    if return_diff > return_threshold:
        predicted_score = 1  # Buy
    elif return_diff < -return_threshold:
        predicted_score = -1  # Sell
    else:
        predicted_score = 0  # Hold
    
    # Step 2: Convert professional recommendation to score
    if 'strong buy' in professional_recommendation.lower():
        professional_score = 2  # Strong Buy
    elif 'buy' in professional_recommendation.lower():
        professional_score = 1  # Buy
    elif 'strong sell' in professional_recommendation.lower():
        professional_score = -2  # Strong Sell
    elif 'sell' in professional_recommendation.lower():
        professional_score = -1  # Sell
    else:
        professional_score = 0  # Hold (if 'hold' or any other neutral recommendation)
    
    # Step 3: Combine the scores
    combined_score = w_predicted * predicted_score + w_professional * professional_score
    
    print("combined_score: ", combined_score)
    # Step 4: Make final recommendation based on combined score
    if combined_score >= 1.29:
        final_recommendation = 'Strong Buy'
    elif combined_score > 0:
        final_recommendation = 'Buy'
    elif combined_score <= -1.29:
        final_recommendation = 'Strong Sell'
    elif combined_score < 0:
        final_recommendation = 'Sell'
    else:
        final_recommendation = 'Hold'
    
    return final_recommendation


# Main function that ties everything together
def get_stock_info(stock_symbol):
    # Fetch financial news for the stock
    news_data = get_today_financial_news(stock_symbol)
    if not news_data:
        print("No news data available.")
        return None
    
    # Analyze sentiment from the news data
    sentiment_vector, confidence_vector = analyze_sentiment(news_data)

    # Calculate cumulative sentiment score and confidence sum
    sentiment_score, confidence_sum = calculate_sentiment_score(sentiment_vector, confidence_vector)

    # Fetch today's stock price
    stock_price_data = get_today_stock_price(stock_symbol)
    if stock_price_data:
        primary_data = stock_price_data['body']['primaryData']
        last_sale_price = primary_data['lastSalePrice']
    else:
        last_sale_price = None, None

    # Fetch historical stock data
    history_price_data = get_history_stock_price(stock_symbol)
    if history_price_data:
        entry = history_price_data['body'][0]
        market_open = entry['regularMarketOpen']
        recommend = entry['averageAnalystRating']
        volume = entry['regularMarketVolume']
    else:
        market_open, recommend, volumn = None, None, None

    # Calculate weighted sentiment score (based on confidence sum)
    weighted_sentiment_score = confidence_sum

    # Predict return
    date = datetime.now().strftime('%Y-%m-%d')
    print("stock_symbol: ", stock_symbol)
    print("date: ", date)
    print("market_open: ", market_open)
    print("volume: ", volume)
    print("weighted_sentiment_score: ", weighted_sentiment_score)
    predicted_return = predict_return(stock_symbol, date, market_open, volume, weighted_sentiment_score)

    price_change = (float(last_sale_price.replace("$", "")) - market_open) / market_open
    print("price_change: ", price_change)
    combined_recommendation = combine_recommendations(price_change, predicted_return, recommend)
    
    recommend = recommend.split(" - ")[1]
    # Return relevant outputs
    return {
        "predicted_return": round(predicted_return, 4),
        "recommend": recommend,
        "last_sale_price": last_sale_price,
        "weighted_sentiment_score": round(weighted_sentiment_score, 4),
        "combined_recommendation": combined_recommendation,
        "open": round(market_open, 4)
    }

# Define the prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    stock_symbol = data.get('stock_symbol')

    # Validate inputs
    if not stock_symbol:
        return jsonify({'error': 'Invalid input. Please provide all required fields.'}), 400

    try:
        # Call the prediction function
        stock_info = get_stock_info(stock_symbol)
        print(stock_info)
        return jsonify({'predicted_return': stock_info['predicted_return'],
                         'recommend': stock_info['recommend'],
                         "last_sale_price": stock_info['last_sale_price'],
                         "weighted_sentiment_score": stock_info['weighted_sentiment_score'],
                         "combined_recommendation": stock_info['combined_recommendation'],
                         "open": stock_info['open']}) 
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.23:5001
Press CTRL+C to quit


today_date:  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date():  2024-12-05
article_date.date()

127.0.0.1 - - [06/Dec/2024 10:10:46] "POST /predict HTTP/1.1" 200 -


stock_symbol:  AAPL
date:  2024-12-06
market_open:  244.05
volume:  36910248
weighted_sentiment_score:  2.811643064022064
date:  2024-12-06
open_price:  <class 'float'>
volume:  <class 'int'>
weighted_sentiment_score:  <class 'float'>
exog:          Date    open    volume  weighted_sentiment_score
0 2024-12-06  244.05  36910248                  2.811643
price_change:  -0.004138496209793154
return_diff:  0.004204281540894435
combined_score:  0.3
{'predicted_return': 0.0001, 'recommend': 'Buy', 'last_sale_price': '$243.04', 'weighted_sentiment_score': 2.8116, 'combined_recommendation': 'Buy', 'open': 244.05}
